In [3]:
import numpy as np
import pandas as pd 
from collections import Counter
import pandas
import csv
from collections import OrderedDict
from operator import itemgetter
from azureml.core import Workspace, Dataset
import random
import requests
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import re

In [4]:
subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'NLP_Project'
workspace_name = 'CougsInAzure'
workspace = Workspace(subscription_id, resource_group, workspace_name)
dataset = Dataset.get_by_name(workspace, name='marx')


In [5]:
df = dataset.to_pandas_dataframe()
df = df.astype(str)
df = df[df['proto'] != 'ICMP']
# Data files from: #https://github.com/danielmiessler/SecLists/blob/master/Usernames/xato-net-10-million-usernames.txt

dateTime= df["datetime"].drop_duplicates().to_list()
host = df["host"].drop_duplicates().to_list()
ip_list = df["srcstr"].to_list()
port_list = df["spt"].apply(lambda x : x.split(".")[0])

In [6]:
dateTime= df["datetime"].drop_duplicates().to_list()
host = df["host"].drop_duplicates().to_list()
ip_list = df["srcstr"].to_list()
port_list = df["spt"].apply(lambda x : x.split(".")[0])

In [7]:
def randomNames(arg):
    switch = {
        1: "_Microsoft",
        2: "&1Apple",
        3: "-21IBM",
        4: "-CougsInAzure",
        5: "_Dell"
    }
    return switch.get(arg, None)

In [8]:
# funtion to transform a pair into a json string
def dataJsonString(location):
    d = "{data:{'hostname':" + location + "}}"
    return str(d)

In [10]:
def falseDataJsonString(num):
    x = random.randint(0,3)
    # Family
    family=open('familyNames.txt')
    familyLines=family.readlines()
    
    # Female
    female=open('femaleNames.txt')
    femaleLines=female.readlines()

    # Male
    male=open('maleNames.txt')
    maleLines=male.readlines()

    # Username
    username=open('usernames.txt')
    usernameLines=username.readlines()

    # Convert data to string
    if x is 0:
        d = "{data:{'hostname':" + maleLines[num] + "-" + familyLines[num] + "}}"
    if x is 1:
        d = "{data:{'hostname':" + femaleLines[num] + "-" + familyLines[num] + "}}"
    if x is 2:
        d = "{data:{'hostname':" + usernameLines[num] + "}}"
    if x is 3:
        d = "{data:{'hostname':" + femaleLines[num] + "-" + familyLines[num] + "}}"

    # Return new non-host string
    return str(d)

In [11]:
data = host
data_len = len(data)-1

datetime_column = []
host_column = []
data_column = []
label_column = []
full_dataset = []

In [12]:
# building the actual column lists
for i in range(0,10000):
    x = random.randint(0,1)
    if x == 1:
        data_column.append(dataJsonString(data[random.randint(0,data_len)]))
        label_column.append(1)
        full_dataset.append([data_column[i], label_column[i]])
    else:
        data_column.append(falseDataJsonString(i))
        label_column.append(0)
        full_dataset.append([data_column[i], label_column[i]])

KeyboardInterrupt: 

In [ ]:
# datetime_column, host_column, and data_column are now filled each with 100,000 items

fdf = pd.DataFrame(full_dataset,columns=['Data', 'Label'])

print("Full Dataset:\n", full_dataset[:5])

In [ ]:
# Determine if nested dictionary
def isDataDict(d):
    if type(d) is not dict:
        return d
    else:
        return isDataDict(d.items())
    return d

In [ ]:
def calcAccuracy(target, predictions):
    total = len(predictions)
    correct = 0
    for i in range(len(predictions)):
        if target[i] == predictions[i]:
            correct += 1
        accuracy = float(correct) / float(total)
    return accuracy

In [ ]:
def main():
    # Find number of times "IP" and "Protocol" appears in the JSON
    index = countIP = countProtocol = countUDP = countTCP = 0
    labels = fdf["Label"].tolist()
    hostData = fdf["Data"].tolist()
    hostData = isDataDict(hostData)

    # Feature lists
    nameFeatureList = []
    lengthFeatureList = []
    dashFeatureList = []
    legalCharList = []
    firstCharList = []    
    completeFeatureList = []
    oneUpper = []
    

    legalChars = set('0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-')
    # Parse the dictionary to find features
    for index in hostData:
        if len(index) > 4:                          # Check length
            lengthFeatureList.append(1)    
        else:
            lengthFeatureList.append(0)
        if "-" in index:                            # Check if dash is used
            dashFeatureList.append(1)    
        else:
            dashFeatureList.append(0)
        if set(index).issubset(legalChars):         # Check if illegal chars are used
            legalCharList.append(1)    
        else:
            legalCharList.append(0)
        if "-" in index[0] or index[0].isnumeric(): # Check is first char is valid
            firstCharList.append(0)    
        else:
            firstCharList.append(1)
        slicedString = index[1:]
        # Check for hostnames that aren't at the beginning of the string.
        if index[0].isupper() and slicedString.islower(): # Check is first character is upper and rest is lower
            oneUpper.append(1)
        else:
            oneUpper.append(0)

    # Combine all feature lists into one
    for i in range(len(firstCharList)):
        completeFeatureList.append([lengthFeatureList[i], dashFeatureList[i], legalCharList[i], firstCharList[i], oneUpper[i]])

    print("List lengths: ", len(lengthFeatureList), len(dashFeatureList), len(legalCharList), len(firstCharList, len(oneUpper)))
    
    print("Complete List: ", completeFeatureList[:5])

    # Train data
    X = completeFeatureList # all the features
    y = labels     # labels, 1 or 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # DECISION TREE CLASSIFIER
    dt = DecisionTreeClassifier().fit(X_train, y_train)
    dtZ = dt.predict(X_test)
    print("Decision Tree: ", calcAccuracy(y_train, dtZ))

    # KNN CLASSIFIER
    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train, y_train)
    knnZ = knn.predict(X_test)
    knnAccuracy = calcAccuracy(y_test, knnZ)
    print("KNN Accuracy: ",knnAccuracy)

    # LINEAR REGRESSION CLASSIFER
    lr = LogisticRegression(random_state=0)
    lr.fit(X_train, y_train)
    lrZ = lr.predict(X_test)
    # calculating accuracy and printing results
    lrAccuracy = calcAccuracy(y_test, lrZ)
    print("Logical Regression Accuracy: ",lrAccuracy)


In [ ]:
main()